# 07.3o: Centroid Survey

**Compute embedding matrix centroids across multiple models**

We've discovered that Qwen2.5 and Qwen3 have black holes (dead vocabulary frozen at initialization) that are OFF-AXIS from the main token cloud centroid. The centroid separation in Qwen2.5-3B is 0.202.

Question: Is this off-center distribution universal, or specific to Qwen? Do other models (Llama, Gemma, earlier Qwen versions) have centroids near the origin, or do they also show this "cosmic wind" bias?

This notebook loads a model's embedding matrix and computes:
1. Centroid location (mean of all token vectors)
2. L2 norm of centroid (distance from origin)
3. L∞ norm of centroid (max component magnitude)

## Parameters

In [25]:
# Model to analyze
MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct"

RANDOM_SEED = 42

## Imports

In [26]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

## Load Model and Extract Embedding Matrix

In [27]:
print(f"Loading model: {MODEL_NAME}\n")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="cpu"
)

# Extract unembedding matrix (lm_head)
gamma = model.lm_head.weight.data.clone().to(torch.float32)
vocab_size, hidden_dim = gamma.shape

print(f"✓ Model loaded")
print(f"Embedding matrix shape: {gamma.shape}")
print(f"Vocabulary size: {vocab_size:,}")
print(f"Hidden dimension: {hidden_dim:,}")

Loading model: unsloth/Llama-3.2-3B-Instruct



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded
Embedding matrix shape: torch.Size([128256, 3072])
Vocabulary size: 128,256
Hidden dimension: 3,072


## Compute Centroid

In [28]:
# Centroid = mean of all token vectors
centroid = gamma.mean(dim=0)

print(f"Centroid statistics:")
print(f"  Shape: {centroid.shape}")
print(f"  Mean component: {centroid.mean().item():.6e}")
print(f"  Std component: {centroid.std().item():.6e}")
print(f"  Min component: {centroid.min().item():.6e}")
print(f"  Max component: {centroid.max().item():.6e}")

Centroid statistics:
  Shape: torch.Size([3072])
  Mean component: -7.738500e-05
  Std component: 7.510118e-03
  Min component: -6.513356e-02
  Max component: 7.285625e-02


## Compute Norms

In [29]:
# L2 norm (Euclidean distance from origin)
l2_norm = torch.norm(centroid, p=2).item()

# L∞ norm (Chebyshev distance from origin, max component magnitude)
l_inf_norm = torch.abs(centroid).max().item()

# L1 norm (Manhattan distance)
l1_norm = torch.abs(centroid).sum().item()

print(f"\nCentroid norms:")
print(f"  L2 (Euclidean): {l2_norm:.6f}")
print(f"  L∞ (Chebyshev): {l_inf_norm:.6e}")
print(f"  L1 (Manhattan): {l1_norm:.6f}")


Centroid norms:
  L2 (Euclidean): 0.416207
  L∞ (Chebyshev): 7.285625e-02
  L1 (Manhattan): 12.792192


## Compare to Typical Token Norms

In [30]:
# Compute L2 norms of all token vectors
token_norms = torch.norm(gamma, p=2, dim=1)

print(f"\nToken L2 norm distribution:")
print(f"  Mean: {token_norms.mean().item():.6f}")
print(f"  Median: {token_norms.median().item():.6f}")
print(f"  Std: {token_norms.std().item():.6f}")
print(f"  Min: {token_norms.min().item():.6f}")
print(f"  Max: {token_norms.max().item():.6f}")

# Centroid as fraction of typical token norm
mean_token_norm = token_norms.mean().item()
centroid_ratio = l2_norm / mean_token_norm

print(f"\nCentroid L2 norm as fraction of mean token norm: {centroid_ratio:.4f}")


Token L2 norm distribution:
  Mean: 1.086129
  Median: 1.090577
  Std: 0.113058
  Min: 0.622125
  Max: 1.469457

Centroid L2 norm as fraction of mean token norm: 0.3832


## Summary

In [31]:
print(f"\n{'='*80}")
print("CENTROID SURVEY SUMMARY")
print(f"{'='*80}")
print(f"Model: {MODEL_NAME}")
print(f"Embedding dimensions: {vocab_size:,} × {hidden_dim:,}")
print()
print(f"Centroid L2 norm: {l2_norm:.6f}")
print(f"Centroid L∞ norm: {l_inf_norm:.6e}")
print(f"Mean token L2 norm: {mean_token_norm:.6f}")
print(f"Centroid / Token ratio: {centroid_ratio:.4f}")
print(f"{'='*80}")


CENTROID SURVEY SUMMARY
Model: unsloth/Llama-3.2-3B-Instruct
Embedding dimensions: 128,256 × 3,072

Centroid L2 norm: 0.416207
Centroid L∞ norm: 7.285625e-02
Mean token L2 norm: 1.086129
Centroid / Token ratio: 0.3832
